In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Convolution2D, MaxPooling2D, Flatten, Dense, Dropout
#from tensorflow.keras.utils import utils
import tensorflow
import matplotlib.pyplot as plt
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Add
#import cv2
#import glob
from keras.preprocessing.image import ImageDataGenerator
 
#---------
 
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import os
from urllib.request import urlopen,urlretrieve
from PIL import Image
from tqdm import tqdm_notebook
%matplotlib inline
from sklearn.utils import shuffle
import cv2
#from resnets_utils import *
 
from keras.models import load_model
from sklearn.datasets import load_files   
from keras.utils import np_utils
from glob import glob
from keras import applications
from keras.preprocessing.image import ImageDataGenerator 
from keras import optimizers
from keras.models import Sequential,Model,load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
from keras.callbacks import TensorBoard,ReduceLROnPlateau,ModelCheckpoint

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
rootDir = r"/content/drive/My Drive/CASIA-Dataset-A/silhouettes"

users = os.listdir(rootDir)

user_path = []
for i in users:
    i = rootDir + "/" + i
    user_path.append(i)

user_path = []
for i in users:
    i = rootDir + "/" + i
    user_path.append(i)
train_set = []
validation_set = []
test_set = []

for user in user_path:
    path = os.listdir(user)
    for num in path:
        if num[-1] == "1" or num[-1] == "2":
            train_set.append(user + "/" + num)
        if num[-1] == "3":
            validation_set.append(user + "/" + num)  
        if num[-1] == "4":
            test_set.append(user + "/" + num)
            

training_path = []
for item in train_set:
    images = os.listdir(item)
    for img in images:
        training_path.append(item + "/" + img)
testing_path = []
for item in test_set:
    images = os.listdir(item)
    for img in images:
        testing_path.append(item + "/" + img)
validation_path = []
for item in validation_set:
    images = os.listdir(item)
    for img in images:
        validation_path.append(item + "/" + img)

In [ ]:
def make_label(img_path):
    label_list = []
    for path in img_path:
        label = (path[52:55])
        label_list.append(label)
    return label_list

In [ ]:
y_col = make_label(training_path)
training_df = pd.DataFrame(list(zip(training_path, y_col)), columns = ["filename", "class"])

y_col_val = make_label(validation_path)
validation_df = pd.DataFrame(list(zip(training_path, y_col)), columns = ["filename", "class"])

## Data Augmentation

In [ ]:
train_datagen = ImageDataGenerator(
                
                shear_range = 0.2,
                zoom_range = 0.2,
                horizontal_flip = True)

test_datagen = ImageDataGenerator()

training_set = train_datagen.flow_from_dataframe(
                dataframe = training_df,
                target_size = (64,64),
                batch_size = 32,
                class_mode = 'categorical')

test_set = test_datagen.flow_from_dataframe(
                dataframe = validation_df,
                target_size = (64,64),
                batch_size = 32,
                class_mode = 'categorical')

Found 9403 validated image filenames belonging to 20 classes.
Found 9403 validated image filenames belonging to 20 classes.


## Model Architecture

In [ ]:
img_height,img_width = 64,64 
num_classes = 20
#If imagenet weights are being loaded
base_model = applications.resnet50.ResNet50(weights='imagenet', include_top=False, input_shape= [img_height,img_width,3])

AttributeError: ignored

In [ ]:
x = base_model.output
#x = GlobalAveragePooling2D()(x)
#x = Dropout(0.7)(x)
x = Flatten()(x)
predictions = Dense(num_classes, activation= 'softmax')(x)
#model.add(Flatten())
model = Model(inputs = base_model.input, outputs = predictions)
model.summary()

NameError: ignored

In [ ]:
from keras.optimizers import SGD, Adam
# sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
adam = Adam(lr=0.0001)
model.compile(optimizer = adam, loss='categorical_crossentropy', metrics=['accuracy'])

## Training Model

In [ ]:
hist = model.fit_generator(training_set, epochs = 10)

Epoch 1/10
  2/294 [..............................] - ETA: 2:21:52 - loss: 4.0519 - accuracy: 0.0469

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 59 could not be retrieved. It could be because a worker has died.
  UserWarning)


  3/294 [..............................] - ETA: 2:23:58 - loss: 4.1786 - accuracy: 0.0312

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 186 could not be retrieved. It could be because a worker has died.
  UserWarning)


 25/294 [=>............................] - ETA: 2:05:52 - loss: 3.8118 - accuracy: 0.0787

KeyboardInterrupt: ignored

## Saving Model

In [ ]:
model.save("model.h5")

In [ ]:
# plt.figure(0)
# plt.plot(hist.history['loss'], 'g')
# plt.plot(hist.history['val_loss'], 'b')

# plt.plot(hist.history['accuracy'],'r')
# plt.plot(hist.history['val_acc'], 'black')
# plt.show()

## Predict Images

In [ ]:
from tensorflow.keras.models import load_model
model = load_model("model.h5")

In [ ]:
from keras.preprocessing import image

test_image = image.load_img(r'C:\Users\Divij\Desktop\wl-00_1-001.png', target_size = (64,64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
training_set.class_indices
result = (result)
print(len(result))

1


In [ ]:
result = result.tolist()
prediction = max(result[0])
person = (result[0].index(prediction))

## Create Dictionary for giving Name to Output

In [ ]:
names = {1:"fyc",
         2:"hy",
         3:"Ijg",
         4:"Iqf",
         5:"Isl",
         6:"ml",
         7:"nhz",
         8:"rj",
         9:"syj",
         10:"wl",
         11:"wq",
         12:"wyc",
         13:"xch",
         14:"xxj",
         15:"yjf",
         16:"zc",
         17:"zdx",
         18:"zjg",
         19:"zl",
         20:"zyf"
        }

In [ ]:
print("The Person is " +  names[person])

The Person is syj


In [ ]:
print(type(x))

<class 'tensorflow.python.framework.ops.Tensor'>
